In [ ]:
import os
import sys
%matplotlib inline
import matplotlib.pyplot as plt
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
from tasks import check

In [ ]:
from model import model58

In [ ]:
all_layers = model58(nb_filters=128, w=32, h=32, c=1, nb_layers=3)

In [ ]:
import theano.tensor as T
import theano
from lasagne import layers

X = T.tensor4()
Y = T.tensor4()


In [ ]:
Y_pred = (layers.get_output(all_layers['output'], X))
loss = ((Y_pred - Y) ** 2).mean()

In [ ]:
from lasagne import updates
import numpy as np
params = layers.get_all_params(all_layers.values(), trainable=True)
print("Number of params : {}".format(layers.count_params(all_layers.values(), trainable=True))) 
lr = theano.shared(np.array(0.01, dtype=theano.config.floatX))
params_updates = updates.momentum(loss, params, learning_rate=lr, momentum=0.9)

In [ ]:
from lasagnekit.misc.draw_net import draw_to_file
from IPython.display import SVG
draw_to_file(layers.get_all_layers(all_layers['output']), 'out.svg')
SVG('out.svg')

In [ ]:
train_fn = theano.function([X, Y], loss, updates=params_updates, on_unused_input='warn')

In [ ]:
predict_fn = theano.function([X], Y_pred, on_unused_input='warn')

In [ ]:
from data import load_data
from lasagnekit.datasets.rescaled import Rescaled
from lasagnekit.easy import iterate_minibatches
from lasagnekit.datasets.imagecollection import ImageCollection
from lasagnekit.datasets.transformed import Transformed
from lasagnekit.datasets.mnist import MNIST
from lasagnekit.datasets.helpers import load_once
from lasagnekit.datasets.subsampled import SubSampled
w, h = 32, 32
scale = {1: 32, 2: 16, 3: 8, 4: 4}
batch_size = 128


data_scale1 = load_once(MNIST)()
data_scale1 = SubSampled(data_scale1, batch_size)
data_scale1 = Rescaled(data_scale1, (w, h))

data_scale2 = Rescaled(data_scale1, (scale[2], scale[2]))
data_scale2 = Rescaled(data_scale2, (scale[1], scale[1]))
c = 1
def preprocess(x, s=32):
    x = x.reshape((x.shape[0], s, s, c))
    x = x.transpose((0, 3, 1, 2))
    return x
data_scale1.load()
data_scale2.load()

In [ ]:
print(all_layers['output'].output_shape)
print(data_scale1.X.shape)
print(data_scale2.X.shape)

In [ ]:
lr.set_value(0.001)
loss_list = []
for i in range(100000):
    data_scale1.load()
    data_scale2.load()
    X_scale1 = preprocess(data_scale1.X, s=scale[1])
    X_scale2 = preprocess(data_scale2.X, s=scale[1])
    total_loss = 0.
    for mini_batch in iterate_minibatches(len(data_scale1.X), batch_size):
        loss_val = train_fn(X_scale2[mini_batch], X_scale1[mini_batch] - X_scale2[mini_batch])
        total_loss += loss_val
    if i % 100 == 0:
        print(total_loss)
    loss_list.append(total_loss)

In [ ]:
plt.plot(loss_list)

In [ ]:
out1 = X_scale2[0:100]
for i in range(1):
    out1 = predict_fn(out1)
#out1 = X_scale1[0:100]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
fig = plt.figure(figsize=(20, 20))
img = tile_raster_images(X_scale1[0:100, 0], (w, h), (10, 10))
plt.imshow(img, interpolation='none', cmap='gray')
fig =plt.figure(figsize=(20, 20))
img = tile_raster_images(out1[0:100, 0] + X_scale2[0:100, 0], (w, h), (10, 10))
plt.imshow(img, interpolation='none', cmap='gray')

In [ ]:
ss = np.random.uniform(size=(100, c, 32, 32))
ss = ss.astype(np.float32)
for i in range(10):
    ss = predict_fn(ss)
print(ss.shape)
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color
fig = plt.figure(figsize=(10, 10))
img = tile_raster_images(ss[0:100].transpose((0, 2, 3, 1)), (w, h), (10, 10))
plt.axis('off')
plt.imshow(img, interpolation='none', cmap='gray')